# Feature selection for all-APGAR model (one hot encoded features)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier

## Load data


In [2]:
data = pd.read_csv('./data/processed_data_for_model_one_hot.csv')

Choose to drop hopsitals from data.

In [3]:
drop_hospitals = True

if drop_hospitals:
    selected_cols = []
    for col in list(data):
        if col[0:8] == 'hospital':
            pass
        else:
            selected_cols.append(col)
    
    data = data[selected_cols]

In [4]:
data.head().T

,0,1,2,3,4
parity,0.0,1.0,1.0,0.0,0.0
prevcaes,0.0,0.0,0.0,0.0,0.0
noscans,1.0,3.0,1.0,2.0,1.0
pyrexia,False,False,False,False,False
meconium,False,False,False,False,False
apgar5,9.0,9.0,10.0,10.0,9.0
bmi,20,25,21,21,0
age,21,22,26,19,29
gest,40,42,38,39,40
ethnic_white,False,False,False,False,False


In [5]:
y = data['apgar5']
X = data.drop('apgar5', axis=1)

In [6]:
# Set up splits
number_of_splits = 3
skf = StratifiedKFold(n_splits = number_of_splits)
skf.get_n_splits(X, y)

3

In [7]:
# Create list to store accuracies and chosen features
roc_auc_by_feature_number = []
roc_auc_by_feature_number_kfold = []
chosen_features = []
best_auc = 0

# Initialise chosen features list and run tracker
available_features = list(X)
number_of_features = len(available_features)

# Loop through number of features
for i in range (20):
    
    # Reset best feature and accuracy
    best_result = 0
    best_feature = ''
    
    # Loop through available features
    for feature in available_features:

        # Create copy of already chosen features to avoid original being changed
        features_to_use = chosen_features.copy()
        # Create a list of features from features already chosen + 1 new feature
        features_to_use.append(feature)
        
        # Set up a list to hold AUC results for this feature for each kfold
        feature_roc_auc_ovr_kfold = []
        
        # Loop through the k-fold splits
        counter = 0
        for train_index, test_index in skf.split(X, y):
            counter += 1
    
            # Get X and Y train/test
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            y_train, y_test = y.loc[train_index], y.loc[test_index]            
            
            # Restrict features
            X_train = X_train[features_to_use]
            X_test = X_test[features_to_use]

            # Define model
            model = XGBClassifier(verbosity = 0, seed=42, learning_rate=0.5)

            # Fit model
            model.fit(X_train, y_train)
            
            # Get target categories from model
            classes = model.classes_

            # Get predicted probabilities
            y_probs = model.predict_proba(X_test)
            
            # Calculate ROC AUC for multiclass models, using One vs Rest
            feature_roc_auc_ovr = roc_auc_score(y_test, y_probs, labels = classes, 
                                multi_class = 'ovr', average = 'macro')
            feature_roc_auc_ovr_kfold.append(feature_roc_auc_ovr)

            # Remove model
            del model
        
        # Get average result from all k-fold splits``
        feature_auc_mean = np.mean(feature_roc_auc_ovr_kfold)
    
        # Update chosen feature and result if this feature is a new best
        if feature_auc_mean > best_result:
            best_result = feature_auc_mean
            best_result_kfold = feature_roc_auc_ovr_kfold
            best_feature = feature
            
    # k-fold splits are complete    
    # Add mean accuracy and AUC to record of accuracy by feature number
    roc_auc_by_feature_number.append(best_result)
    roc_auc_by_feature_number_kfold.append(best_result_kfold)
    chosen_features.append(best_feature)
    available_features.remove(best_feature)
            
    print (f'Feature {i+1:2.0f}: {best_feature}, AUC: {best_result:0.3f}')
    
    # Check if AUC is still increasinbg. Stop if not
    #if best_result > best_auc:
    #    best_auc = best_result
    #else:
    #    print ('AUC falling; selection stopped')
    #    break            

Feature  1: gest, AUC: 0.625
Feature  2: ctg_A, AUC: 0.689
Feature  3: analgesia_G, AUC: 0.716
Feature  4: meconium, AUC: 0.719
Feature  5: delivery_B, AUC: 0.721
Feature  6: induction_O, AUC: 0.724
Feature  7: delivery_A, AUC: 0.726
Feature  8: analgesia_T, AUC: 0.726
Feature  9: delivery_R, AUC: 0.727
Feature 10: analgesia_P, AUC: 0.728
Feature 11: ethnic_white, AUC: 0.728
Feature 12: analgesia_O, AUC: 0.728
Feature 13: delivery_O, AUC: 0.730
Feature 14: analgesia_S, AUC: 0.731
Feature 15: onset_E, AUC: 0.731
Feature 16: analgesia_C, AUC: 0.730
Feature 17: onset_N, AUC: 0.731
Feature 18: delivery_E, AUC: 0.731
Feature 19: pyrexia, AUC: 0.732
Feature 20: ctg_N, AUC: 0.732


In [9]:
s = pd.Series(chosen_features)
s

0             gest
1            ctg_A
2      analgesia_G
3         meconium
4       delivery_B
5      induction_O
6       delivery_A
7      analgesia_T
8       delivery_R
9      analgesia_P
10    ethnic_white
11     analgesia_O
12      delivery_O
13     analgesia_S
14         onset_E
15     analgesia_C
16         onset_N
17      delivery_E
18         pyrexia
19           ctg_N
dtype: object

In [12]:
s.to_csv('./output/feature_selection_full_model.csv', header=False)